In [1]:
import os, random, glob
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential, Model, load_model
from keras.layers import GlobalAveragePooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('tf')

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ImportError: cannot import name ReduceLROnPlateau

In [4]:
from skimage.data import imread
from skimage.io import imshow,imsave
from skimage import img_as_float
import pandas as pd
import numpy as np
import cv2
from skimage.util import crop
from skimage.transform import rotate
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline
import math

In [2]:
TRAIN_DIR = '../data/train/'
TEST_DIR = '../data/test_stg1/'
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
modelStr = 'Crop'
ROWS = 224
COLS = 224
BatchSize = 64
LearningRate = 1e-4
le = LabelEncoder()
le.fit(FISH_CLASSES)
le.transform(FISH_CLASSES)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [13]:
def deg_angle_between(x1,y1,x2,y2):
    from math import atan2, degrees, pi
    dx = x2 - x1
    dy = y2 - y1
    rads = atan2(dy,dx)
    rads %= 2*pi
    degs = degrees(rads)
    return(degs)

u'img_00003.jpg'

In [14]:
def get_rotated_cropped_fish(img,x1,y1,x2,y2):
    (h,w) = img.shape[:2]
    #calculate center and angle
    center = ( (x1+x2) / 2,(y1+y2) / 2)
    angle = np.floor(deg_angle_between(x1,y1,x2,y2))
    print('angle=' +str(angle) + ' ')
    print('center=' +str(center))
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))
    
    fish_length = np.sqrt((x1-x2)**2+(y1-y2)**2)
    cropped = rotated[(max((center[1]-fish_length/1.8),0)):(max((center[1]+fish_length/1.8),0)) ,
                      (max((center[0]- fish_length/1.8),0)):(max((center[0]+fish_length/1.8),0))]
    imshow(img)
    imshow(rotated)
    imshow(cropped)
    resized = resize(cropped,(ROWS,COLS))
    return(resized)

[{u'class': u'point', u'x': 825.5028464692997, u'y': 342.8499725255559},
 {u'class': u'point', u'x': 1095.1227277758048, u'y': 449.36646884417524}]

In [12]:
label_files = ['/home/nati/data/kaggle/fish/input/train/BET/bet_labels.json',
             '/home/nati/data/kaggle/fish/input/train/ALB/alb_labels.json',
             '/home/nati/data/kaggle/fish/input/train/YFT/yft_labels.json',
             '/home/nati/data/kaggle/fish/input/train/DOL/dol_labels.json',
             '/home/nati/data/kaggle/fish/input/train/SHARK/shark_labels.json',
             '/home/nati/data/kaggle/fish/input/train/LAG/lag_labels.json',
             '/home/nati/data/kaggle/fish/input/train/OTHER/other_labels.json']

data_dirs = ['/home/nati/data/kaggle/fish/input/train/BET/',
             '/home/nati/data/kaggle/fish/input/train/ALB/',
             '/home/nati/data/kaggle/fish/input/train/YFT/',
             '/home/nati/data/kaggle/fish/input/train/DOL/',
             '/home/nati/data/kaggle/fish/input/train/SHARK/',
             '/home/nati/data/kaggle/fish/input/train/LAG/',
             '/home/nati/data/kaggle/fish/input/train/OTHER/']

,annotations,class,filename
1,"[{u'y': 342.849972526, u'x': 825.502846469, u'...",image,img_00003.jpg


In [5]:
images = list()
labels_list = list()
for c in range(7):
    labels = pd.read_json(label_files[c])
    for i in range(len(labels)):
        try:
            img_filename = labels.iloc[i,2]
            print(img_filename)
            l1 = pd.DataFrame((labels[labels.filename==img_filename].annotations).iloc[0])
            image = imread(data_dirs[c]+img_filename)
            images.append(get_rotated_cropped_fish(image,np.floor(l1.iloc[0,1]),np.floor(l1.iloc[0,2]),np.floor(l1.iloc[1,1]),np.floor(l1.iloc[1,2])))
            print('success')
            labels_list.append(c)
        except:
            pass

,annotations,class,filename
0,"[{u'y': 155.0, u'x': 409.0, u'class': u'point'}]",image,image2.jpg
1,"[{u'y': 342.849972526, u'x': 825.502846469, u'...",image,img_00003.jpg
2,"[{u'y': 606.090168807, u'x': 721.615642971, u'...",image,img_00010.jpg
3,"[{u'y': 514.004645922, u'x': 492.238976876, u'...",image,img_00012.jpg
4,"[{u'y': 359.55, u'x': 258.03, u'class': u'poin...",image,img_00015.jpg
5,"[{u'y': 459.2511, u'x': 165.0123, u'class': u'...",image,img_00019.jpg
6,"[{u'y': 675.171053793, u'x': 712.942860998, u'...",image,img_00020.jpg
7,"[{u'y': 458.25, u'x': 713.46, u'class': u'poin...",image,img_00029.jpg
8,"[{u'y': 553.344341256, u'x': 526.033354456, u'...",image,img_00032.jpg
9,"[{u'y': 280.234473254, u'x': 307.545460054, u'...",image,img_00037.jpg


In [3]:
le.transform(['LAG', 'YFT', 'OTHER', 'DOL', 'SHARK', 'NoF', 'BET', 'ALB'])

array([3, 7, 5, 2, 6, 4, 1, 0])

In [3]:
#Loading data

import pickle

def get_images(fish):
    """Load files from train folder"""
    fish_dir = TRAIN_DIR+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

def read_image(src):
    """Read and resize individual images"""
    im = Image.open(src)
    im = im.resize((COLS, ROWS), Image.BILINEAR)
    im = np.asarray(im)
    return im
    
if os.path.exists('../data/data_train_{}_{}.pickle'.format(ROWS, COLS)):
    print ('Exist data_train_{}_{}.pickle. Loading data from file.'.format(ROWS, COLS))
    with open('../data/data_train_{}_{}.pickle'.format(ROWS, COLS), 'rb') as f:
        data_train = pickle.load(f)
    X_train = data_train['X_train']
    y_train = data_train['y_train']
else:
    print ('Loading data from original images. Generating data_train_{}_{}.pickle.'.format(ROWS, COLS))

    files = []
    y_train = []

    for fish in FISH_CLASSES:
        fish_files = get_images(fish)
        files.extend(fish_files)

        y_fish = np.tile(fish, len(fish_files))
        y_train.extend(y_fish)
        #print("{0} photos of {1}".format(len(fish_files), fish))

    y_train = np.array(y_train)
    X_train = np.ndarray((len(files), ROWS, COLS, 3), dtype=np.uint8)

    for i, im in enumerate(files): 
        X_train[i] = read_image(TRAIN_DIR+im)
        if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

    #X_train = X_train / 255.
    #print(X_train.shape)

    # One Hot Encoding Labels
    y_train = le.transform(y_train)
    y_train = np_utils.to_categorical(y_train)

    #save data to file
    data_train = {'X_train': X_train,'y_train': y_train }

    with open('../data/data_train_{}_{}.pickle'.format(ROWS, COLS), 'wb') as f:
        pickle.dump(data_train, f)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=None, stratify=y_train)

Exist data_train_224_224.pickle. Loading data from file.


In [4]:
#data preprocessing

train_datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rescale=1./255,
    rotation_range=180,
    shear_range=np.pi/6.,
    zoom_range=[1,1.1],
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)

#train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BatchSize, shuffle=True, seed=None)

valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow(X_valid, y_valid, batch_size=BatchSize, shuffle=True, seed=None)

In [5]:
#callbacks

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')        

model_checkpoint = ModelCheckpoint(filepath='./checkpoints/weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
        
learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)


In [ ]:
#stg1 training

from keras.applications.vgg16 import VGG16

optimizer = Adam(lr=LearningRate)

base_model = VGG16(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, init='glorot_normal', activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dense(256, init='glorot_normal', activation='relu')(x)
#x = Dropout(0.5)(x)
predictions = Dense(len(FISH_CLASSES), init='glorot_normal', activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(train_generator, samples_per_epoch=len(X_train), nb_epoch=300, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=valid_generator, nb_val_samples=len(X_valid), nb_worker=3, pickle_safe=True)

Epoch 1/300
3008/3021 [============================>.] - ETA: 0s - loss: 1.7936 - acc: 0.3873

/opt/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_loss improved from 1.54322 to 1.53578, saving model to ./checkpoints/weights.000-1.5358.hdf5
3072/3021 [==============================] - 82s - loss: 1.7922 - acc: 0.3867 - val_loss: 1.5358 - val_acc: 0.4714
Epoch 2/300
3072/3021 [==============================] - 65s - loss: 1.5797 - acc: 0.4619 - val_loss: 1.5301 - val_acc: 0.4714
Epoch 3/300
3047/3021 [==============================] - 67s - loss: 1.6069 - acc: 0.4411 - val_loss: 1.5064 - val_acc: 0.4766
Epoch 4/300
3072/3021 [==============================] - 66s - loss: 1.5391 - acc: 0.4580 - val_loss: 1.5418 - val_acc: 0.4466
Epoch 5/300
3072/3021 [==============================] - 65s - loss: 1.5339 - acc: 0.4626 - val_loss: 1.4179 - val_acc: 0.5221
Epoch 6/300
3047/3021 [==============================] - 66s - loss: 1.4901 - acc: 0.4851 - val_loss: 1.4232 - val_acc: 0.4922
Epoch 7/300
3072/3021 [==============================] - 66s - loss: 1.4769 - acc: 0.5029 - val_loss: 1.4101 - val_acc: 0.5286
Epoch 8/300
3

In [24]:
#stg2 training

from keras.applications.vgg16 import VGG16

optimizer = Adam(lr=LearningRate)

base_model = VGG16(weights='imagenet', include_top=False)
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:14]:
   layer.trainable = False
for layer in model.layers[14:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(train_generator, samples_per_epoch=len(X_train), nb_epoch=300, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=valid_generator, nb_val_samples=len(X_valid), nb_worker=3, pickle_safe=True)

0 input_3
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
Epoch 1/300
3008/3021 [============================>.] - ETA: 0s - loss: 1.2538

/opt/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_loss improved from inf to 1.04548, saving model to ./checkpoints/weights.000-1.0455.hdf5
3072/3021 [==============================] - 85s - loss: 1.2478 - val_loss: 1.0455
Epoch 2/300
3072/3021 [==============================] - 70s - loss: 0.9437 - val_loss: 0.8057
Epoch 3/300
3047/3021 [==============================] - 71s - loss: 0.7734 - val_loss: 0.9774
Epoch 4/300
3072/3021 [==============================] - 70s - loss: 0.6909 - val_loss: 0.6304
Epoch 5/300
3072/3021 [==============================] - 70s - loss: 0.5813 - val_loss: 0.7017
Epoch 6/300
3047/3021 [==============================] - 69s - loss: 0.5596 - val_loss: 0.6150
Epoch 7/300
3072/3021 [==============================] - 70s - loss: 0.4975 - val_loss: 0.5357
Epoch 8/300
3072/3021 [==============================] - 70s - loss: 0.4361 - val_loss: 0.5432
Epoch 9/300
3047/3021 [==============================] - 69s - loss: 0.4032 - val_loss: 0.5752
Epoch 10/300
3072/3021 [===========================

In [ ]:
#resume training

files = glob.glob('./checkpoints/*')
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
index = val_losses.index(min(val_losses))
print('Loading model from checkpoints file ' + files[index])
model = load_model(files[index])

model.fit_generator(train_generator, samples_per_epoch=len(X_train), nb_epoch=300, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=valid_generator, nb_val_samples=len(X_valid), nb_worker=3, pickle_safe=True)

Loading model from checkpoints file ./checkpoints/weights.028-1.2077.hdf5
Epoch 1/300
3008/3021 [============================>.] - ETA: 0s - loss: 1.2688 - acc: 0.5455

/opt/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_loss improved from inf to 1.20833, saving model to ./checkpoints/weights.000-1.2083.hdf5
3072/3021 [==============================] - 82s - loss: 1.2664 - acc: 0.5472 - val_loss: 1.2083 - val_acc: 0.5938
Epoch 2/300
 832/3021 [=======>......................] - ETA: 37s - loss: 1.2594 - acc: 0.5601

In [38]:
#test submission

import datetime

if os.path.exists('../data/data_test_{}_{}.pickle'.format(ROWS, COLS)):
    print ('Exist data_test_{}_{}.pickle. Loading test data from file.'.format(ROWS, COLS))
    with open('../data/data_test_{}_{}.pickle'.format(ROWS, COLS), 'rb') as f:
        data_test = pickle.load(f)
    X_test = data_test['X_test']
    test_files = data_test['test_files']
else:
    print ('Loading test data from original images. Generating data_test_{}_{}.pickle.'.format(ROWS, COLS))

    test_files = [im for im in os.listdir(TEST_DIR)]
    X_test = np.ndarray((len(test_files), ROWS, COLS, 3), dtype=np.uint8)

    for i, im in enumerate(test_files): 
        X_test[i] = read_image(TEST_DIR+im)
        if i%300 == 0: print('Processed {} of {}'.format(i, len(test_files)))
            
    data_test = {'X_test': X_test,'test_files': test_files }
    
    with open('../data/data_test_{}_{}.pickle'.format(ROWS, COLS), 'wb') as f:
        pickle.dump(data_test, f)
            
X_test = X_test / 255.

files = glob.glob('./checkpoints/*')
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
index = val_losses.index(min(val_losses))
model = load_model(files[index])

test_preds = model.predict(X_test, batch_size=BatchSize, verbose=1)
#test_preds= test_preds / np.sum(test_preds,axis=1,keepdims=True)

submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
#submission.loc[:, 'image'] = pd.Series(test_files, index=submission.index)
submission.insert(0, 'image', test_files)

now = datetime.datetime.now()
info = modelStr + '{:.4f}'.format(min(val_losses))
sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
submission.to_csv(sub_file, index=False)

Exist data_test_224_224.pickle. Loading test data from file.
1000/1000 [==============================] - 28s    
